In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
from keras.layers import Input, Dense, Convolution2D, MaxPooling2D, UpSampling2D, Reshape, Flatten
from keras.models import Model
from IPython.display import SVG
from keras.utils.visualize_util import model_to_dot
from keras.callbacks import ModelCheckpoint

import os
import random
import keras


Using TensorFlow backend.


In [18]:
ROW = 80
COL = 80
CHANNELS = 3

TRAIN_DIR = 'cat_dog_train/'
TEST_DIR = 'cat_dog_test/'

train_images = [TRAIN_DIR+i for i in os.listdir(TRAIN_DIR)] # use this for full dataset

def readImg(imgFile):
    colorImg = cv2.imread(imgFile, cv2.IMREAD_COLOR)
    colorImg = cv2.cvtColor(colorImg, cv2.COLOR_BGR2RGB)
    colorImg = cv2.resize(colorImg, (ROW, COL))/255.0
    
    greyImg = cv2.imread(imgFile, cv2.IMREAD_GRAYSCALE)
    greyImg = cv2.cvtColor(greyImg,cv2.COLOR_GRAY2RGB)
    greyImg = cv2.resize(greyImg, (ROW, COL))/255.0
    
    return greyImg,colorImg
    
    

def generateDate(imgFiles):
    count = len(imgFiles)
    dataX = np.ndarray((count, ROW, COL,CHANNELS), dtype=float)
    dataY = np.ndarray((count, ROW, COL,CHANNELS), dtype=float)
    for i, image_file in enumerate(imgFiles):
        gImg,cImg = readImg(image_file)
        dataX[i] = gImg
        dataY[i] = cImg  
    return dataX,dataY

random.shuffle(train_images)


import math
def chunked(iterable, n):
    chunksize = int(math.ceil(len(iterable) / n))
    return (iterable[i * chunksize:i * chunksize + chunksize]
            for i in range(n))

BATCHNUMBER= 20

chuckedTrainList = list(chunked(train_images,BATCHNUMBER))






In [22]:
BaseLevel = ROW////2
input_img = Input(shape=(ROW,COL,CHANNELS))
x = Convolution2D(256, 5, 5, activation='relu', border_mode='same')(input_img)
x = MaxPooling2D((2, 2), border_mode='same')(x)
x = Convolution2D(128, 5, 5, activation='relu', border_mode='same')(x)
x = MaxPooling2D((2, 2), border_mode='same')(x)
x = Flatten()(x)

encoded = Dense(2000)(x)
oneD = Dense(BaseLevel*BaseLevel*128)(encoded)
fold = Reshape((BaseLevel,BaseLevel,128))(oneD)

x = UpSampling2D((2, 2))(fold)
x = Convolution2D(128, 5, 5, activation='relu', border_mode='same')(x)
x = UpSampling2D((2, 2))(x)
decoded = Convolution2D(3, 5, 5, activation='sigmoid', border_mode='same')(x)

autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')


SyntaxError: invalid syntax (<ipython-input-22-691912db50a0>, line 1)

In [24]:
checkpoint = ModelCheckpoint(filepath="/home/yhfy2006/machineLearningInPython/AutoEncoder/logs/weights.hdf5", verbose=1, save_best_only=True)
earlyStopping=keras.callbacks.EarlyStopping(monitor='val_loss', patience=0, verbose=0, mode='auto')

cbks = [checkpoint,earlyStopping]


digitalImg = dict()
epoch = 500

for ii in range(epoch):
    i = ii%len(chuckedTrainList)
    print("epoch =====>"+str(ii))
    imgfiles = chuckedTrainList[i]
    if str(i) in digitalImg:
        dataX = digitalImg[str(i)][0]
        dataY = digitalImg[str(i)][1]
    else:
        dataX,dataY = generateDate(imgfiles)
        digitalImg[str(i)] = (dataX,dataY)
        
    autoencoder.fit(dataX, dataY,
                nb_epoch=1,
                batch_size=50,
                shuffle=True,
                verbose=2,
                validation_split=0.3,
                callbacks=cbks)
        

    



In [15]:
#autoencoder.load_weights("logs/weights_best_20170301.hdf5")

In [28]:
test_images =  [TEST_DIR+i for i in os.listdir(TEST_DIR)]
test_images =  test_images[80:92]

testX,testY = generateDate(test_images)

predict = autoencoder.predict(testX)

for i,imageFile in enumerate(test_images):
    plt.imshow(testX[i])
    plt.show()
    
    plt.imshow(predict[i].astype(float))
    plt.show()
    
    plt.imshow(testY[i])
    plt.show()

    


In [144]:
autoencoder.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_21 (InputLayer)            (None, 500, 500, 3)   0                                            
____________________________________________________________________________________________________
convolution2d_133 (Convolution2D (None, 500, 500, 16)  448         input_21[0][0]                   
____________________________________________________________________________________________________
maxpooling2d_58 (MaxPooling2D)   (None, 250, 250, 16)  0           convolution2d_133[0][0]          
____________________________________________________________________________________________________
convolution2d_134 (Convolution2D (None, 250, 250, 8)   1160        maxpooling2d_58[0][0]            
___________________________________________________________________________________________